# ROOT files
This document deals with ROOT files only (measured at V20 in February 2018). It displays metadata stored in these files and it offers a choice of output formats for the user:
- 1D datasets: 
    - ASCII files (.dat) 
    - .png  files  
- 2D datasets:
    - ASCII files (.dat)
    - .png files
    - .tiff files
    
For the 2D plots, there is the option of inverting the `y-axis`.

**Information about data in ROOT files**

| Spectrum     | Folders with data to plot | Sample   | Duration |
| :----------- |:--------------------------| :------- | :------- |
| Spectrum3    | Meas_3;1                  | NAK      | 13 hours |
| Spectrum8    | Meas_2;1                  | Si-NIST  | 19 hours |
| Spectrum9    | Meas_1;1                  | Y2O3     | 22 hours |
| Spectrum11   | Meas_1;1                  | NAK      | 22 hours | 
| Spectrum12   | Meas_1;1                  | NAK      | 22 hours |

In [ ]:
import os
import uproot
import matplotlib.pyplot as plt
import numpy as np

import dataconfig  # to get paths to data

# to save tiff files
from PIL import Image

# for interactive elements
import ipywidgets

# to remove problems with too many figures open
plt.rcParams.update({'figure.max_open_warning': 0})

%matplotlib widget

# Display metadata stored in a ROOT file
The information related to only one ROOT file is displayed. 


In [ ]:
assert os.path.exists(dataconfig.data_root), 'The path to ROOT files does not exist.'

# Create dictionary to generate plots
# Each entry corresponds to the spectrum number, the associated root file and the folder to get the data from
dict_root_files = {'Spectrum03': ('Spectrum03_DENEX006_1_18-02-05_0000.root', 'Meas_3'), 
                   'Spectrum08': ('Spectrum08_DENEX006_1_18-02-07_0002.root', 'Meas_2'),
                   'Spectrum09': ('Spectrum09_DENEX006_1_18-02-08_0001.root', 'Meas_1'),
                   'Spectrum11': ('Spectrum11_DENEX006_1_18-02-09_0001.root', 'Meas_1'),
                   'Spectrum12': ('Spectrum12_DENEX006_1_18-02-10_0000.root', 'Meas_1')}

In [ ]:
# Display metadata stored in ROOT files stored in previously defined dictionary
for key_spectrum in dict_root_files.keys():
    
    print(50*'-','\n',key_spectrum)
    
    ROOTfile, dir_with_data = dict_root_files[key_spectrum]
    
    file_to_open = os.path.join(dataconfig.data_root, ROOTfile)
    
    assert os.path.isfile(file_to_open), 'There is an issue with the file to be opened.'

    with uproot.open(file_to_open)[dir_with_data] as myFile:
        
        for key in myFile.keys():
            if 'BoardParam_run' in str(key):
                myObject = myFile[key]
                nb_xbins = myObject.numbins
                
                for i in range(nb_xbins):
                    print(f"{myObject.xlabels[i]}: {myObject.values[i]}")
                # save metadata in file 
                output_file_metadata = ROOTfile.split('.')[0] + 'BoardParam_run.dat'
                with open(output_file_metadata, "w") as output_file:
                    np.savetxt(output_file, np.c_[myObject.xlabels, myObject.values], fmt=['%s','%s'])  

# Plot all graphs stored in ROOT files and save as .dat, .png or .tiff files
This process is done for ROOT files related to Spectrum03, 08, 09, 11, and 12.

In [ ]:
invert_y = ipywidgets.Checkbox(
    value=True,
    description='Invert y-axis of 2D plots for ROOT files?',
    layout=ipywidgets.Layout(width='350px'))

invert_y

In [ ]:
save_1d_data_to_asciifiles = ipywidgets.Checkbox(
    value=True,
    description='Save 1D data to ascii files?',
    layout=ipywidgets.Layout(width='350px'))

save_1d_data_to_png = ipywidgets.Checkbox(
    value=True,
    description='Save 1D data to png files?',
    layout=ipywidgets.Layout(width='350px'))

save_2d_data_to_asciifiles = ipywidgets.Checkbox(
    value=True,
    description='Save 2D data to ascii files?',
    layout=ipywidgets.Layout(width='350px'))

save_2d_data_to_png = ipywidgets.Checkbox(
    value=True,
    description='Save 2D data to png files?',
    layout=ipywidgets.Layout(width='350px'))

save_2d_data_to_tiff = ipywidgets.Checkbox(
    value=True,
    description='Save 2D data to tiff files?',
    layout=ipywidgets.Layout(width='350px'))

box_saving_selection = ipywidgets.HBox([
    ipywidgets.VBox([save_1d_data_to_asciifiles, save_1d_data_to_png]), 
    ipywidgets.VBox([save_2d_data_to_asciifiles,
                     save_2d_data_to_png,
                     save_2d_data_to_tiff])])

box_saving_selection

In [ ]:
# loop over selected root files
for key_spectrum in dict_root_files.keys():
    print(50*'-','\n',key_spectrum)
    
    ROOTfile = dict_root_files[key_spectrum][0]
    dir_with_data = dict_root_files[key_spectrum][1]
    
    # open ROOT file 
    file_to_open = os.path.join(dataconfig.data_root, ROOTfile)
    
    assert os.path.isfile(file_to_open), 'There is an issue with the file to be opened.'

    with uproot.open(file_to_open)[dir_with_data] as myFile:
        for key in myFile.keys():
            # do not consider problematic dataset in Spectrum09
            if key_spectrum=="Spectrum09" and ('1D3' in str(key) or 'FINISHED' in str(key)):
                continue
            
            if 'TH1I' in str(myFile[key]):
                key_name = myFile[key].name.decode('utf-8')
                arr_object = myFile[key].values
                
                # naming of outputs - the extension will be added once the format 
                # is chosen (.png or .dat for 1D data)  
                name_output_file = ROOTfile[:10] + key_name.replace(',','_')
    
                # plot
                fig1, ax1 = plt.subplots()
                ax1.plot(arr_object)
                ax1.set_title(key_name) 
                fig1.canvas.header_visible = False
                fig1.canvas.toolbar_position = 'bottom'
                  
                # to save png plot 
                if save_1d_data_to_png.value:
                    fig1.savefig(name_output_file + '.png')
            
                # to save ascii files
                if save_1d_data_to_asciifiles.value:
                    np.savetxt(name_output_file + '.dat', arr_object, fmt= "%d")
                        
            # 2D contourplot
            elif 'TH2' in str(myFile[key]):
                key_name = myFile[key].name.decode('utf-8') 
                
                # extract info about x, y axis (min, max and number of bins)
                x_min = myFile[key].xlow 
                x_max = myFile[key].xhigh 
                bins_x = myFile[key].xnumbins 
                y_min = myFile[key].ylow 
                y_max = myFile[key].yhigh 
                bins_y = myFile[key].ynumbins 
                    
                # create x- and y-axis
                xaxis = x_min + (x_max - x_min) / (bins_x - 1) * np.arange(bins_x)
                yaxis = y_min + (y_max - y_min) / (bins_y - 1) * np.arange(bins_y)
                
                # invert y axis
                if invert_y.value:
                    arr_object = np.flip(myFile[key].values, 1)
                    # add info about inverted y axis to name of outputs - the extension will be added later
                    name_output_file = ROOTfile[:10] + "_" + key_name.replace(',','_') + "_inv_y"
                # keep original orientation
                else:
                    arr_object = myFile[key].values
                    # name of outputs - the extension will be added later
                    name_output_file = ROOTfile[:10] + "_" + key_name.replace(',','_')
                    
                # plot
                fig2, ax2 = plt.subplots()
                CS = ax2.contourf(xaxis, yaxis, arr_object.transpose(), cmap=plt.cm.get_cmap('gist_earth'))
                ax2.set_title(key_name)
                cbar = fig2.colorbar(CS)
                fig2.canvas.header_visible = False
                fig2.canvas.toolbar_position = 'bottom'
                    
                if save_2d_data_to_asciifiles.value:
                    np.savetxt(name_output_file + '.dat', arr_object, fmt= "%d")
                 
                # save 2d in png files
                if save_2d_data_to_png.value:
                    fig2.savefig(name_output_file + '.png')
                        
                if save_2d_data_to_tiff.value:
                    if not save_2d_data_to_png.value:
                        print('.png files are required to generate .tiff files')
                        break
                    else:
                        im_from_png = np.array(Image.open(name_output_file + '.png'))
                        Image.fromarray(im_from_png).save(name_output_file + ".tiff", "TIFF")   